# Lecture 7

## Structure of a Python file

All your Python files should look like this (minus the optional parts):

In [ ]:
# Imports at the top
import sklearn


# Global variables next, uppercase
NEIGHBORS_KNN = 2


# Functions come next.
def square(num):
    """
    This is a doc-string. It's the triple-quoted string right after a function
    definition. It contains doc-tests. For now, you don't to write anything here
    other than doc-tests.

    These are doc-tests, starting with triple angled brackets and a space.
    The line after the triple-quoted bracket is the expected result.
    Corner cases should use the syntax below for the "traceback" of the error.
    >>> square(2)
    4
    >>> square("2")
    Traceback (most recent call last):
    ...
    ValueError: argument must be a number
    """

    if not isinstance(width, (int, float)):
        raise ValueError("argument must be a number")

    return num ** 2


# Classes come next, if any.
class MyClass(object):
    pass


# A main function comes next, if any.
def main():
    pass


# Finally, if you want to run code when the file is run but not when it's imported,
# use the "dunder" variable `__name__`, which is then equal to "__main__":
if __name__ == "__main__":
    main()

## New style standard: lines under 80 characters

Function and variable names are often descriptive and long.

All these examples are equivalent, so use them to get under 80 characters:

In [ ]:
a = some_very_long_function_name(some_very_long_parameter_name, another_very_long_parameter_name)

a = some_very_long_function_name(some_very_long_parameter_name,
                                 another_very_long_parameter_name)

a = some_very_long_function_name(
    some_very_long_parameter_name,
    another_very_long_parameter_name)



# TODO: find the bug in this code

A leap year is a year that is divisible by 4 and not by 100, unless also by 400



In [ ]:
def is_leap(year):
    return (year % 4) and not ((year % 100) and not (year % 400))


# A buggy function from the morning midterm (exercise 3)

Here, we take a string and return a list of lines containing the same contents, but where each line has 80 characters or less. This is called "refilling". Some common sense rules are:

- only split a word across two list elements if it's over 80 characters long. Otherwise, split it at a the last space in the first 80 characters
- do not return a space at the start of an element, because it looks ugly.

You should write a recursive "helper" function that does this work, and a non-recursive "wrapper" function that handles the edge cases and calls the helper/recursive function. Gradescope will ONLY call the wrapper function.

In order to write short doc-tests, the functions also take an argument `max_chars`, set at 80 by default. This way, you do not need to write 80 characters in the doc-tests to check the functionality.

Try to find the bug in this student submission, which splits a line of 80 characters when it should not:

In [ ]:
SEPARATOR = " "

def refill_recursive_student(s, max_chars):
    # Note: this function should have no doc-tests.
    str_list = s.split(SEPARATOR)
    if '' in str_list:
        str_list.remove('')
    long_list = []
    temp_list = []
    count = 0
    for word in str_list:
        length = len(word) + 1
        if count <= max_chars - length:
            temp_list.append(word)
            count += length
        else:
            line = SEPARATOR.join(temp_list)
            long_list.append(line)
            temp_list = []
            count = length
            temp_list.append(word)
    line = SEPARATOR.join(temp_list)
    long_list.append(line)
    return long_list

result = refill_recursive_student("scrambled shares. The earsplitting policeman repeats. The fertile faucet sleeps.", 80)
print(result)
print(len(result[0] + SEPARATOR + result[1]))

['scrambled shares. The earsplitting policeman repeats. The fertile faucet', 'sleeps.']
80


## Speed

### Avoid loops

Loops can be slow in Python. Python is optimized so that built-in functions and operations run a lot faster than loops.

For example, consider this example ([source](https://medium.com/@tententgc/vectorization-vs-loops-the-secret-to-massive-python-performance-gains-af8a4ac17234)), which computes the sum of integers up to 1.5 million:


In [ ]:
import time


N = int(1.5e6)


start = time.time()

total = 0
for item in range(N):
    total += item

end = time.time()

print(round(end - start, 3), "seconds")
# 0.208-0.529 seconds


0.377 seconds


If instead we use the built-in function `sum()`, the code becomes 4-5 times faster:

In [ ]:
import time


N = int(1.5e6)


start = time.time()
total = sum(range(N))
end = time.time()

print(round(end - start, 3), "seconds")
# 0.046-0.063 seconds


0.042 seconds


NumPy is optimized further because it compiles down to C, and code doing the same logic is 26 times faster than a loop:


In [ ]:
import time
import numpy as np


N = int(1.5e6)


start = time.time()
total = np.sum(np.arange(N))
end = time.time()

print(round(end - start, 3), "seconds")
# 0.008-0.012 seconds


0.016 seconds


### Don't check arguments inside a recursive function

If using recursion, don't check arguments and defensive programming in the recursive function. Instead, write a wrapper function whose only purpose is to check arguments once, and then call a recursive function that does not check arguments and runs fast.

Here is an example from the afternoon midterm (exercise 17):

> The first function calculates the sum of all elements in a list of integers using recursion. The second function is a "wrapper" around this first function. It should have doc-tests, handle edges cases, and then call the first function.

In a recursion with a list of size `N` if we check for the type of the argument, and of the first element inside the recursion, it's slow.

We can use a wrapper function to check it once, then call the recursive function. Then, the code runs 2 times faster:

In [ ]:
import time

def calculate_sum_recursive_slow(a):

    if not isinstance(a, list):
        raise ValueError("argument must be a list")

    if 0 == len(a):
        return 0

    if not isinstance(a[0], (int, float)):
        raise ValueError("the list must have numbers only")

    return a[0] + calculate_sum_recursive(a[1:])


def calculate_sum_recursive(a):

    if 0 == len(a):
        return 0

    return a[0] + calculate_sum_recursive(a[1:])


def calculate_sum(a):

    if not isinstance(a, list):
        raise ValueError("argument must be a list")

    for i in a:
        if not isinstance(a[0], (int, float)):
            raise ValueError("the list must have numbers only")

    return calculate_sum_recursive(a)


N = int(9e2)
a = list(range(N))
start = time.time()
calculate_sum_recursive_slow(a)
print(time.time() - start)


start = time.time()
calculate_sum(a)
print(time.time() - start)

0.010812044143676758
0.0038046836853027344
